In [ ]:
import numpy
import pytorch_lightning
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
class Perceptron(pytorch_lightning.LightningModule):
    def __init__(self):
        super().__init__()
        # Define the single linear layer (4 input features, 1 output)
        self.layer = nn.Linear(4, 1)

    def forward(self, x):
        # Pass the input through the linear layer
        # then through the sigmoid activation function
        return torch.relu(self.layer(x))

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.binary_cross_entropy(y_hat, y.view(-1, 1).float())
        self.log("train_loss", loss)  # Logging the loss
        return loss

    def configure_optimizers(self):
        # We will use the Adam optimizer
        return torch.optim.Adam(self.parameters(), lr=0.001)

In [ ]:
# Generate synthetic data for the example
numpy.random.seed(42)
N = 1000
temp = numpy.random.uniform(0, 100, N)
rain = numpy.random.randint(0, 2, N)
ride = numpy.random.randint(0, 2, N)
friends = numpy.random.randint(0, 2, N)
X = numpy.vstack([temp, rain, ride, friends]).T
# Assume that the individual will go to the zoo if there is a ride and friends are going
Y = (ride & friends).astype(numpy.float32)

# Split the data into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# Create TensorDatasets and DataLoaders
train_data = TensorDataset(
    torch.tensor(X_train, dtype=torch.float32),
    torch.tensor(Y_train, dtype=torch.float32),
)
val_data = TensorDataset(
    torch.tensor(X_val, dtype=torch.float32), torch.tensor(Y_val, dtype=torch.float32)
)
train_loader = DataLoader(train_data, batch_size=32, num_workers=4)
val_loader = DataLoader(val_data, batch_size=32, num_workers=4)

In [ ]:
# Start TensorBoard.
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

In [ ]:
# Train the model
model = Perceptron()
trainer = pytorch_lightning.Trainer(max_epochs=50, log_every_n_steps=3)
trainer.fit(model, train_loader)

In [ ]:
# Generate some new data to test
test_temp = numpy.array([75, 85, 60, 90, 72])
test_rain = numpy.array([0, 1, 0, 1, 0])
test_ride = numpy.array([1, 0, 1, 0, 1])
test_friends = numpy.array([1, 1, 0, 0, 1])
X_test = numpy.vstack([test_temp, test_rain, test_ride, test_friends]).T

# Convert to a PyTorch tensor
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# Make predictions
with torch.no_grad():
    predictions = model(X_test_tensor)

# Print predictions
for i, pred in enumerate(predictions):
    print(
        f"Conditions: {X_test[i]} | Probability of going to the zoo: {pred.item():.4f}"
    )